## Prepare Data for ViT-based estimation method

used code in ViT-pytorch/

In [30]:
import argparse
import datetime
import json
import random
import time
from pathlib import Path
import os, sys
import numpy as np
import torch
from tools.utils_at import *
from mmdet.models.utils import transform_tensors_to_list
from mmdet.structures.bbox import (bbox_cxcywh_to_xyxy, bbox_overlaps,
                                   bbox_xyxy_to_cxcywh)
from torchvision.ops.boxes import box_area
import torch.nn.functional as F

# function
def box_iou(boxes1, boxes2):
    area1 = box_area(boxes1)
    area2 = box_area(boxes2)


    lt = torch.max(boxes1[:, None, :2], boxes2[:, :2])  # [N,M,2]
    rb = torch.min(boxes1[:, None, 2:], boxes2[:, 2:])  # [N,M,2]

    wh = (rb - lt).clamp(min=0)  # [N,M,2]
    inter = wh[:, :, 0] * wh[:, :, 1]  # [N,M]

    union = area1[:, None] + area2 - inter

    iou = inter / (union + 1e-6)
    return iou, union

def generalized_box_iou(boxes1, boxes2):
    """
    Generalized IoU from https://giou.stanford.edu/

    The boxes should be in [x0, y0, x1, y1] format

    Returns a [N, M] pairwise matrix, where N = len(boxes1)
    and M = len(boxes2)
    """
    # degenerate boxes gives inf / nan results
    # so do an early check
    assert (boxes1[:, 2:] >= boxes1[:, :2]).all()
    assert (boxes2[:, 2:] >= boxes2[:, :2]).all()

    iou, union = box_iou(boxes1, boxes2)

    lt = torch.min(boxes1[:, None, :2], boxes2[:, :2])
    rb = torch.max(boxes1[:, None, 2:], boxes2[:, 2:])

    wh = (rb - lt).clamp(min=0)  # [N,M,2]
    area = wh[:, :, 0] * wh[:, :, 1]

    return iou - (area - union) / (area + 1e-6)
    
def hungarian_matching(out_logits, out_boxes, tgt_ids, tgt_bbox, cost_class_weight = 1.0, cost_bbox_weight = 5.0, cost_giou_weight = 2.0, focal_alpha = 0.25):
    """ Performs the matching
    """
    
    # We flatten to compute the cost matrices in a batch
    num_queries = out_logits.shape[0]
    out_prob = out_logits.softmax(dim=1)  # [num_queries, num_classes]
    
    # Compute the classification cost.
    alpha = focal_alpha
    gamma = 2.0
    neg_cost_class = (1 - alpha) * (out_prob ** gamma) * (-(1 - out_prob + 1e-8).log())
    pos_cost_class = alpha * ((1 - out_prob) ** gamma) * (-(out_prob + 1e-8).log())
    cost_class = pos_cost_class[:, tgt_ids] - neg_cost_class[:, tgt_ids]
    
    # Compute the L1 cost between boxes
    cost_bbox = torch.cdist(out_boxes, tgt_bbox, p=1)
    
    # Compute the giou cost betwen boxes            
    cost_giou = -generalized_box_iou(bbox_cxcywh_to_xyxy(out_boxes), bbox_cxcywh_to_xyxy(tgt_bbox))
    
    # Final cost matrix
    C = cost_bbox * cost_bbox_weight + cost_class * cost_class_weight + cost_giou * cost_giou_weight
    C = C.view(num_queries, -1)
    result = torch.argmin(C, axis=1)
    return result

def get_indexes(out_logits):
    prob = out_logits.softmax(dim=1)
    prob, _ = prob.max(dim=1)
    select_mask = prob > score_threshold
    if sum(select_mask) == 0:
        score, indexes = prob.topk(10)
        indexes,_ = torch.sort(indexes, dim=0)
        print(f"Cannot find detected objects with score larger than {score_threshold}")
    else:
        indexes = select_mask.nonzero().reshape(-1)
    return indexes

def sigmoid_focal_loss(inputs, targets, alpha: float = 0.25, gamma: float = 2):
    """
    Loss used in RetinaNet for dense detection: https://arxiv.org/abs/1708.02002.
    Args:
        inputs: A float tensor of arbitrary shape.
                The predictions for each example.
        targets: A float tensor with the same shape as inputs. Stores the binary
                 classification label for each element in inputs
                (0 for the negative class and 1 for the positive class).
        alpha: (optional) Weighting factor in range (0,1) to balance
                positive vs negative examples. Default = -1 (no weighting).
        gamma: Exponent of the modulating factor (1 - p_t) to
               balance easy vs hard examples.
    Returns:
        Loss tensor
    """
    prob = inputs.sigmoid()
    ce_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction="none")
    p_t = prob * targets + (1 - prob) * (1 - targets)
    loss = ce_loss * ((1 - p_t) ** gamma)

    if alpha >= 0:
        alpha_t = alpha * targets + (1 - alpha) * (1 - targets)
        loss = alpha_t * loss
    return loss
    
def compute_loss(out_logits, out_boxes, tgt_ids, tgt_bbox, matched_target_indexes, cls_loss_coef = 1.0, bbox_loss_coef = 5.0, giou_loss_coef = 2.0):
    if matched_target_indexes == None:
        target_classes_onehot = torch.zeros([out_logits.shape[0], out_logits.shape[1]],dtype=out_logits.dtype, layout=out_logits.layout, 
                                        device=out_logits.device)
        loss_ce = sigmoid_focal_loss(out_logits, target_classes_onehot)
        loss_ce = loss_ce.sum(axis=1)
        loss = loss_ce * cls_loss_coef
        return loss, loss_ce, torch.zeros(loss.shape), torch.zeros(loss.shape)
    cls_loss_coef = 1.0
    bbox_loss_coef = 5.0
    giou_loss_coef = 2.0
    target_boxes = tgt_bbox[matched_target_indexes]
    loss_bbox = F.l1_loss(out_boxes, target_boxes, reduction='none') # [num_queries, 4]
    loss_bbox = loss_bbox.mean(axis=1) # [num_queries]
    loss_giou = 1 - torch.diag(generalized_box_iou(bbox_cxcywh_to_xyxy(out_boxes),
                bbox_cxcywh_to_xyxy(target_boxes))) # [num_queries]
    target_classes_onehot = torch.zeros([out_logits.shape[0], out_logits.shape[1]],dtype=out_logits.dtype, layout=out_logits.layout, 
                                        device=out_logits.device)
    target_labels = torch.tensor(tgt_ids)[matched_target_indexes]
    target_classes_onehot.scatter_(1, target_labels.unsqueeze(-1), 1)
    loss_ce = sigmoid_focal_loss(out_logits, target_classes_onehot)
    loss_ce = loss_ce.sum(axis=1)
    loss = loss_ce * cls_loss_coef + loss_bbox * bbox_loss_coef + loss_giou * giou_loss_coef
    return loss, loss_ce, loss_bbox, loss_giou

def generate_feature_annotation(num, count):
    file_path = data_path + str(num) + ".json"
    output_data = read_json_results(file_path)
    feature = np.array(output_data['feature'][5][0])
    out_logits = torch.FloatTensor(output_data['pred_logits'][5][0])
    out_boxes = torch.FloatTensor(output_data['pred_boxes'][5][0])
    target_labels = output_data['gt_labels']
    target_boxes = torch.FloatTensor(output_data['gt_boxes'])
    if len(target_boxes) == 0:
        return False
    target_boxes = bbox_xyxy_to_cxcywh(target_boxes)
    img_h, img_w = output_data['img_metas'][0]['img_shape']
    factors = output_data['img_metas'][0]['scale_factor']
    shapes = torch.FloatTensor([img_h, img_w, img_h, img_w])
    target_boxes = target_boxes / shapes
    out_logits = out_logits[:, 0:80] # for detr & coco
    indexes = get_indexes(out_logits)
    out_logits = out_logits[indexes]
    out_boxes = out_boxes[indexes]
    gt_indexes = hungarian_matching(out_logits, out_boxes, target_labels, target_boxes)
    loss, loss_ce, loss_bbox, loss_giou = compute_loss(out_logits, out_boxes, target_labels, target_boxes, gt_indexes)
    np_write(feature, feature_path + str(count) + ".npy")
    json_data = {}
    json_data['loss'] = transform_tensors_to_list(loss)
    json_data['loss_ce'] = transform_tensors_to_list(loss_ce)
    json_data['loss_bbox'] = transform_tensors_to_list(loss_bbox)
    json_data['loss_giou'] = transform_tensors_to_list(loss_giou)
    json_data['index'] = transform_tensors_to_list(indexes)
    json_data['gt_indexes'] = transform_tensors_to_list(gt_indexes)
    json_data['output_file_num'] = num
    write_json_results(json_data, annotation_path + str(count) + ".json")
    return True

In [31]:
split = "val"
base_path = "./pro_data/detr/"
data_path = base_path + split + "/outputs/"
feature_path = base_path + split + "/feature/"
annotation_path = base_path + split + "/annotation/"
create_folder_if_not_exists(feature_path)
create_folder_if_not_exists(annotation_path)
score_threshold = 0.98

### Generate feature and annotation

In [ ]:
files_in_folder = os.listdir(data_path)
num_files = len(files_in_folder) - 1
count = 0
for num in range(num_files):
    flag = generate_feature_annotation(num, count)
    if flag:
        count += 1
    if (num+1)%1000 == 0:
        print(num)

### Generate image_true_losses and region_true_losses

In [24]:
files_in_folder = os.listdir(annotation_path)
nums = len(files_in_folder) - 1
img_loss = np.zeros(nums)
region_loss = []
for i in range(nums):
    path = annotation_path + str(i) + ".json"
    json_data = read_json_results(path)
    losses = np.array(json_data['loss'])
    img_loss[i] = losses.mean()
    region_loss.extend(json_data['loss'])
np_write(img_loss, base_path + split + "/image_true_losses.npy")
np_write(np.array(region_loss), base_path + split + "/region_true_losses.npy")

In [36]:
split = "train"
image_loss = np_read(base_path + split + "/image_true_losses.npy")
print(image_loss.min(), image_loss.max())
region_loss = np_read(base_path + split + "/region_true_losses.npy")
print(region_loss.min(), region_loss.max())

1.308374007542928 34.88671875
0.07327228039503098 37.063392639160156


In [34]:
num = 25
file_path = data_path + str(num) + ".json"
output_data = read_json_results(file_path)

In [ ]:
feature = np.array(output_data['feature'][5][0])
out_logits = torch.FloatTensor(output_data['pred_logits'][5][0])
out_boxes = torch.FloatTensor(output_data['pred_boxes'][5][0])
target_labels = output_data['gt_labels']
target_boxes = torch.FloatTensor(output_data['gt_boxes'])
if len(target_boxes) > 0:
    target_boxes = bbox_xyxy_to_cxcywh(target_boxes)
img_h, img_w = output_data['img_metas'][0]['img_shape']
factors = output_data['img_metas'][0]['scale_factor']
shapes = torch.FloatTensor([img_h, img_w, img_h, img_w])
target_boxes = target_boxes / shapes
out_logits = out_logits[:, 0:80] # for detr & coco
indexes = get_indexes(out_logits)
out_logits = out_logits[indexes]
out_boxes = out_boxes[indexes]
gt_indexes = hungarian_matching(out_logits, out_boxes, target_labels, target_boxes)
loss, loss_ce, loss_bbox, loss_giou = compute_loss(out_logits, out_boxes, target_labels, target_boxes, gt_indexes)

In [37]:
len(target_boxes)

0

In [14]:
np_write(feature, feature_path + str(num) + ".npy")
json_data = {}
json_data['loss'] = transform_tensors_to_list(loss)
json_data['loss_ce'] = transform_tensors_to_list(loss_ce)
json_data['loss_bbox'] = transform_tensors_to_list(loss_bbox)
json_data['loss_giou'] = transform_tensors_to_list(loss_giou)
json_data['index'] = transform_tensors_to_list(indexes)
json_data['gt_indexes'] = transform_tensors_to_list(gt_indexes)
json_data['output_file_num'] = num
write_json_results(json_data, annotation_path + str(num) + ".json")

In [67]:
json = read_json_results(base_path + "train" + "/annotation/" + "55397.json")

In [68]:
json['output_file_num']

55894

In [60]:
patch_loss = torch.tensor(json['loss'])
img_loss = patch_loss.mean()
patch_index = torch.tensor(json['index'])
annotation = torch.full((100,), 255, dtype=patch_loss.dtype)
annotation[patch_index] = patch_loss

In [61]:
annotation

tensor([255.0000, 255.0000,   7.3523, 255.0000, 255.0000, 255.0000, 255.0000,
        255.0000, 255.0000, 255.0000, 255.0000,   4.2870, 255.0000, 255.0000,
        255.0000, 255.0000, 255.0000, 255.0000, 255.0000,   5.5946, 255.0000,
          8.4145,   5.0837, 255.0000, 255.0000, 255.0000, 255.0000, 255.0000,
        255.0000, 255.0000, 255.0000, 255.0000,   5.2658,   5.4688,   5.0664,
        255.0000, 255.0000,   5.0659, 255.0000,   7.0068, 255.0000, 255.0000,
        255.0000, 255.0000, 255.0000, 255.0000, 255.0000, 255.0000, 255.0000,
        255.0000, 255.0000,   6.5768, 255.0000, 255.0000, 255.0000, 255.0000,
        255.0000, 255.0000, 255.0000, 255.0000, 255.0000, 255.0000,   8.6178,
        255.0000, 255.0000, 255.0000, 255.0000, 255.0000, 255.0000, 255.0000,
          4.8617, 255.0000, 255.0000,   6.8172, 255.0000, 255.0000, 255.0000,
        255.0000,   6.6019,   6.3947,  11.1883, 255.0000, 255.0000, 255.0000,
        255.0000, 255.0000, 255.0000, 255.0000, 255.0000, 255.00

In [62]:
patch_index

tensor([ 2, 11, 19, 21, 22, 32, 33, 34, 37, 39, 51, 62, 70, 73, 78, 79, 80, 90,
        94])